# In this notebook we use the imagenet ResNet50 to predict dogs and cats

**We will be using Keras with its tensorflow backend**

*We also use some sklearn functionalities*

In [1]:
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from keras.applications.resnet50 import preprocess_input
from keras.applications import ResNet50
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LeakyReLU
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from IPython.display import display
from PIL import Image

Using TensorFlow backend.


**We saved the weights of the ResNet50 in a folder**

We also used a toy test folder where we saved manually images from google and tested them with our model.

*We saved our data in the following architecture:*

//

resnet50Transfer

input
    - resnet50
          - resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
          - resnet50_weights_tf_dim_ordering_tf_kernels.h5
    - train
          - dogs
              - dog.1.jpg
              - dog.2.jpg
              - ...
          - cats
              - cat.1.jpg
              - cat.2.jpg
              - ...
    - val
          - dogs
              - dog.1.jpg
              - dog.2.jpg
              - ...
          - cats
              - cat.1.jpg
              - cat.2.jpg
              - ...
    - test
          - test
              - 1.jpg
              - 2.jpg
              - ...
    - test_it
          - dogs
              - dog.1.jpg
              - dog.2.jpg
              - ...
          - cats
              - cat.1.jpg
              - cat.2.jpg
              - ...
//

In [2]:
resnet_weights_path = "input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"

## With big data we need to create data generators

**It allows us to *batchify*, *shuffle* and specify automatically our X_train, Y_train with categories we build in our data architecture**

Initially Wrapping your head around them is annoying but it makes training easier and faster.

In [ ]:
# create a datagenerator that preprocesses inputs for ResNet50
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

# these functions help to process big amounts of data
# each subdirectory of images is a category /dogs is 1 /cats is 0
# we specify the resizing of the images to 224x224
# we can apply mini batch to better process our learning
# finally we specify the color mode and the class mode
train_generator = data_generator.flow_from_directory(
        'input/train',
        batch_size=24, shuffle=False,
        target_size=(512, 512),
        color_mode = "rgb",
        class_mode='categorical')

val_generator = data_generator.flow_from_directory(
        'input/val',
        batch_size=1, shuffle=False,
        target_size=(512, 512),
        color_mode = "rgb",
        class_mode='categorical')

test_generator = data_generator.flow_from_directory(
        'input/test',
        batch_size = 1,
        target_size=(512, 512),
        color_mode = "rgb",
        class_mode='categorical')

test_it_generator = data_generator.flow_from_directory(
        'input/test_it',
        batch_size = 1, shuffle = True,
        target_size=(512, 512),
        color_mode = "rgb",
        class_mode='categorical')

In [4]:
val_generator.class_indices
train_generator.class_indices

{'cats': 0, 'dogs': 1}

### Here we create our model using Resnet50 as the first layer

In [82]:
# clear Keras session if environment already used
K.clear_session()

# Init sequential model
model = Sequential()

# initialize first layer with already trained ResNet50, specifying inputshapes and methods of pooling
model.add(ResNet50(include_top=False, 
                   pooling='max', input_shape = (512, 512, 3),
                   weights=resnet_weights_path))

# add our untrained layer for dog cat prediction
model.add(Dense(4, activation="relu"))

model.add(Dense(2, activation= "softmax"))

# specify keras not to train ResNet50
model.layers[0].trainable = False

# compile model with adam & categorical crossentropy
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

# get summary of model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 8196      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 10        
Total params: 23,595,918
Trainable params: 8,206
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
# we fit our model with our train generator functions
# it will be functional very fast
model.fit_generator(
        train_generator,
        epochs = 2,
        steps_per_epoch=10,
        validation_data=val_generator ,
        validation_steps=15 ,
        verbose= 1
)

We just used transfer learning for basic cat/dog classification.
For this we used the pretrained model we downloaded from Kaggle: https://www.kaggle.com/keras/resnet50

We organized our data with Bash using regex, we downloaded our data comme from the competition: https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition

Eventually we create a data generator specifying image resizing, target mode and fit our model with our data gen.

Since we use ResNet50: https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/He_Deep_Residual_Learning_CVPR_2016_paper.pdf trained on ImageNet: http://www.image-net.org/ we reach a very high score very rapidly.

In [85]:
# saving the weights
model.save('epic_model.h5')

In [8]:
# loading the weights
model.load_weights('epic_model.h5')

In [88]:
# evaluating the model
loss, acc = model.evaluate_generator(val_generator, steps=20)
(loss, acc)

(1.1920928955078125e-07, 1.0)